In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split

from config import *
from helpers import *

### Filter function for vulnerability commits

In [29]:
def filter_commits(commits):
    """ wip """
    label= []
    global STATS

    for index, commit in commits.iterrows():
        STATS["commits"] += 1

        if not filter_title(CONFIG, get_commit_title(commit)):
            label.append(0)
            continue
        if any(hit in get_commit_title(commit) for hit in ["Merge", "Revert"]):
            label.append(0)
            continue
        STATS["fixes"] += 1

        hits = filter_commit(CONFIG, commit)
        if not hits:
            label.append(0)
            continue
        STATS["filtered"] += 1

        reporter = get_commit_reporter(commit['commit_msg'])
        CVEs = get_commit_cves(commit['commit_msg']) # enough to not check title?

        if not filter_reporter(CONFIG, reporter):
            label.append(0)
            continue
        else:
            label.append(1)

    return label

### Load and Process dataset

In [30]:
df = pd.read_csv('data/bigvul_devign_cvefixes_neuralsentry_commits.csv')
df = df.drop_duplicates(subset=['commit_hash'],keep='first')
df = df.dropna(subset=['commit_msg', 'commit_hash'])
train_df, df = train_test_split(df, test_size=0.2, random_state=420)
    
linux_test_df = pd.read_csv('data/linux_test.csv')
vulnfix = linux_test_df[linux_test_df['labels'] == 1]
non_vulnfix = linux_test_df[linux_test_df['labels'] == 0]
linux_test_df = pd.concat([non_vulnfix, vulnfix])

### Prediction on BigVul, Devign, CVEfixes and NeuralSentry Dataset

In [31]:
predictions = filter_commits(df)
labels = df['labels'].tolist()
generate_metrics(labels, predictions)


>>> Confusion Matrix: 
TP: 62, FP: 1 
FN: 3493, TN: 3442 

>>> Accuracy:  0.500714489854244 

>>>: Classification Report:
               precision    recall  f1-score   support

  non-bugfix       0.50      1.00      0.66      3443
      bugfix       0.98      0.02      0.03      3555

    accuracy                           0.50      6998
   macro avg       0.74      0.51      0.35      6998
weighted avg       0.74      0.50      0.34      6998



### Prediction on Linux Kernel CVEs Dataset

In [32]:
predictions = filter_commits(linux_test_df)
labels = linux_test_df['labels'].tolist()
generate_metrics(labels, predictions)


>>> Confusion Matrix: 
TP: 136, FP: 1 
FN: 1691, TN: 502 

>>> Accuracy:  0.2738197424892704 

>>>: Classification Report:
               precision    recall  f1-score   support

  non-bugfix       0.23      1.00      0.37       503
      bugfix       0.99      0.07      0.14      1827

    accuracy                           0.27      2330
   macro avg       0.61      0.54      0.26      2330
weighted avg       0.83      0.27      0.19      2330

